In [2]:
import torch
import torch.nn.functional as F

In [8]:
with open('./names.txt','r',encoding='utf-8') as f:
    words = f.read().splitlines()

In [4]:
words[:4]

['emma', 'olivia', 'ava', 'isabella']

In [5]:
list(zip(words[:4],words[1:5]))

[('emma', 'olivia'),
 ('olivia', 'ava'),
 ('ava', 'isabella'),
 ('isabella', 'sophia')]

In [10]:
chars = []
chars.append('.')
chars = chars+sorted(list(set(''.join(words))))

In [11]:
# 定义两个函数用以转换字符到数字和数字到字符
stoi = lambda x: chars.index(x)
itos = lambda x: chars[x]

print(stoi('a'))
print(itos(0))

1
.


In [8]:
for w in words[:3]:
    chs = '.'+ w + '.'
    for c1, c2 in zip(chs, chs[1:]):
        print(c1, c2)

. e
e m
m m
m a
a .
. o
o l
l i
i v
v i
i a
a .
. a
a v
v a
a .


In [9]:
F.one_hot(torch.tensor([0,1, 2, 3]), 4)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

In [56]:
x = []
y = []
for w in words[:3]:
    tx = []
    ty = []
    chs = '.'+ w + '.'
    for c1, c2 in zip(chs, chs[1:]):
        tx.append(c1)
        ty.append(c2)
    x.append(tx)
    y.append(ty)
print(x)
print(y)

[['.', 'e', 'm', 'm', 'a'], ['.', 'o', 'l', 'i', 'v', 'i', 'a'], ['.', 'a', 'v', 'a']]
[['e', 'm', 'm', 'a', '.'], ['o', 'l', 'i', 'v', 'i', 'a', '.'], ['a', 'v', 'a', '.']]


In [57]:
xenc = [[stoi(x) for x in xi] for xi in x]
yenc = [[stoi(y) for y in yi] for yi in y]

In [58]:
g = torch.Generator().manual_seed(2147483647)

In [59]:
X = F.one_hot(torch.tensor(xenc[0]),num_classes=len(chars)).float()
Y = F.one_hot(torch.tensor(yenc[0]),num_classes=len(chars)).float()

In [60]:
w = torch.randn((27,27),generator=g,requires_grad=True)

In [15]:
b = torch.randn((1,27),generator=g)

In [61]:
logits = X @ w

In [62]:
logits

tensor([[ 1.5674e+00, -2.3729e-01, -2.7385e-02, -1.1008e+00,  2.8588e-01,
         -2.9643e-02, -1.5471e+00,  6.0489e-01,  7.9136e-02,  9.0462e-01,
         -4.7125e-01,  7.8682e-01, -3.2843e-01, -4.3297e-01,  1.3729e+00,
          2.9334e+00,  1.5618e+00, -1.6261e+00,  6.7716e-01, -8.4039e-01,
          9.8488e-01, -1.4837e-01, -1.4795e+00,  4.4830e-01, -7.0730e-02,
          2.4968e+00,  2.4448e+00],
        [ 4.7236e-01,  1.4830e+00,  3.1748e-01,  1.0588e+00,  2.3982e+00,
          4.6827e-01, -6.5650e-01,  6.1662e-01, -6.2197e-01,  5.1007e-01,
          1.3563e+00,  2.3445e-01, -4.5585e-01, -1.3132e-03, -5.1161e-01,
          5.5570e-01,  4.7458e-01, -1.3867e+00,  1.6229e+00,  1.7197e-01,
          9.8846e-01,  5.0657e-01,  1.0198e+00, -1.9062e+00, -4.2753e-01,
         -2.1259e+00,  9.6041e-01],
        [ 1.9359e-01,  1.0532e+00,  6.3393e-01,  2.5786e-01,  9.6408e-01,
         -2.4855e-01,  2.4756e-02, -3.0404e-02,  1.5622e+00, -4.4852e-01,
         -1.2345e+00,  1.1220e+00, -6.73

In [64]:
logits.shape

torch.Size([5, 27])

In [48]:
prob = torch.exp(logits) /(torch.sum(torch.exp(logits),dim=1,keepdim=True))

In [49]:
torch.sum(prob,dim=1,keepdim=True)

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SumBackward1>)

In [50]:
prob[0]

tensor([0.0128, 0.0224, 0.0241, 0.0642, 0.0173, 0.0180, 0.0050, 0.0151, 0.0512,
        0.0321, 0.0615, 0.0525, 0.0466, 0.0595, 0.0031, 0.0107, 0.1154, 0.0364,
        0.0177, 0.0151, 0.0795, 0.0151, 0.0354, 0.1211, 0.0171, 0.0271, 0.0239],
       grad_fn=<SelectBackward0>)

In [22]:
Y[0]

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
prob[0] * Y[0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0123, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [39]:
prob[0,yenc[0][0]]

tensor(0.0123)

In [51]:
r = prob * Y

In [52]:
rs = torch.sum(r, dim=1)

In [53]:
nll =-torch.log(rs)

In [54]:
nll.mean()

tensor(3.5329, grad_fn=<MeanBackward0>)

In [5]:
# all the data
g = torch.Generator().manual_seed(2147483647)
x = []
y = []
for w in words:
    tx = []
    ty = []
    chs = '.'+ w + '.'
    for c1, c2 in zip(chs, chs[1:]):
        tx.append(c1)
        ty.append(c2)
    x.append(tx)
    y.append(ty)
print(len(x))
print(len(y))

32033
32033


In [55]:
nll.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [11]:
xenc = [[stoi(x) for x in xi] for xi in x]
yenc = [[stoi(y) for y in yi] for yi in y]

In [14]:
w = torch.randn((27,27),generator=g,requires_grad=True)

for i in range(len(words)):
    X = F.one_hot(torch.tensor(xenc[i]),num_classes=len(chars)).float()
    Y = F.one_hot(torch.tensor(yenc[i]),num_classes=len(chars)).float()
    logits = X @ w
    prob = torch.exp(logits) /(torch.sum(torch.exp(logits),dim=1,keepdim=True))
    r_prob = prob * Y
    r_prob_sum = torch.sum(r_prob, dim=1)
    nll =-torch.log(r_prob_sum)
    loss = torch.mean(nll)
    print(loss)
    w.grad = None
    loss.backward()
    w.grad.data = w.grad.data - 0.1*w.grad.data

tensor(3.5329, grad_fn=<MeanBackward0>)
tensor(3.5149, grad_fn=<MeanBackward0>)
tensor(3.6020, grad_fn=<MeanBackward0>)
tensor(3.6209, grad_fn=<MeanBackward0>)
tensor(3.5735, grad_fn=<MeanBackward0>)
tensor(3.7092, grad_fn=<MeanBackward0>)
tensor(3.7474, grad_fn=<MeanBackward0>)
tensor(3.6659, grad_fn=<MeanBackward0>)
tensor(3.9338, grad_fn=<MeanBackward0>)
tensor(3.7189, grad_fn=<MeanBackward0>)
tensor(4.0534, grad_fn=<MeanBackward0>)
tensor(3.7423, grad_fn=<MeanBackward0>)
tensor(4.2539, grad_fn=<MeanBackward0>)
tensor(3.5322, grad_fn=<MeanBackward0>)
tensor(3.5877, grad_fn=<MeanBackward0>)
tensor(3.5498, grad_fn=<MeanBackward0>)
tensor(3.6912, grad_fn=<MeanBackward0>)
tensor(3.9666, grad_fn=<MeanBackward0>)
tensor(3.5274, grad_fn=<MeanBackward0>)
tensor(4.2999, grad_fn=<MeanBackward0>)
tensor(3.1942, grad_fn=<MeanBackward0>)
tensor(3.7976, grad_fn=<MeanBackward0>)
tensor(3.2307, grad_fn=<MeanBackward0>)
tensor(4.0781, grad_fn=<MeanBackward0>)
tensor(3.8353, grad_fn=<MeanBackward0>)


In [ ]:
# all the data
g = torch.Generator().manual_seed(2147483647)
x = []
y = []
for w in words:
    chs = '.'+ w + '.'
    for c1, c2 in zip(chs, chs[1:]):
        x.append(stoi(c1))
        y.append(stoi(c2))
   
print(x[:5])
print(y[:5])

[0, 5, 13, 13, 1]
[5, 13, 13, 1, 0]


In [25]:
len(x)

228146

In [27]:
X = F.one_hot(torch.tensor(x),num_classes=len(chars)).float()
Y = F.one_hot(torch.tensor(y),num_classes=len(chars)).float()

In [41]:
X.shape

torch.Size([228146, 27])

In [ ]:
w = torch.randn((27,27),generator=g,requires_grad=True)

In [49]:
for i in range(100):
    logits = X @ w
    prob = torch.exp(logits) /(torch.sum(torch.exp(logits),dim=1,keepdim=True))
    r_prob = prob * Y
    r_prob_sum = torch.sum(r_prob, dim=1)
    nll =-torch.log(r_prob_sum)
    loss = torch.mean(nll)
    print(loss)
    w.grad = None
    loss.backward()
    w.data = w.data - 50*w.grad.data


tensor(2.4622, grad_fn=<MeanBackward0>)
tensor(2.4621, grad_fn=<MeanBackward0>)
tensor(2.4621, grad_fn=<MeanBackward0>)
tensor(2.4620, grad_fn=<MeanBackward0>)
tensor(2.4620, grad_fn=<MeanBackward0>)
tensor(2.4619, grad_fn=<MeanBackward0>)
tensor(2.4619, grad_fn=<MeanBackward0>)
tensor(2.4618, grad_fn=<MeanBackward0>)
tensor(2.4618, grad_fn=<MeanBackward0>)
tensor(2.4617, grad_fn=<MeanBackward0>)
tensor(2.4617, grad_fn=<MeanBackward0>)
tensor(2.4617, grad_fn=<MeanBackward0>)
tensor(2.4616, grad_fn=<MeanBackward0>)
tensor(2.4616, grad_fn=<MeanBackward0>)
tensor(2.4615, grad_fn=<MeanBackward0>)
tensor(2.4615, grad_fn=<MeanBackward0>)
tensor(2.4615, grad_fn=<MeanBackward0>)
tensor(2.4614, grad_fn=<MeanBackward0>)
tensor(2.4614, grad_fn=<MeanBackward0>)
tensor(2.4613, grad_fn=<MeanBackward0>)
tensor(2.4613, grad_fn=<MeanBackward0>)
tensor(2.4613, grad_fn=<MeanBackward0>)
tensor(2.4612, grad_fn=<MeanBackward0>)
tensor(2.4612, grad_fn=<MeanBackward0>)
tensor(2.4611, grad_fn=<MeanBackward0>)


In [ ]:
# sample
x = stoi('.')
x_input = F.one_hot(torch.tensor(x), len(chars)).float()
y_pred_logits = x_input @ w
y_pred_logits.shape

torch.Size([27])

In [102]:
y_pred_prob = torch.exp(y_pred_logits)/torch.sum(torch.exp(y_pred_logits))

In [112]:
y_pred_prob

tensor([0.1960, 0.0164, 0.0160, 0.0139, 0.0308, 0.0204, 0.0040, 0.0050, 0.0688,
        0.0487, 0.0052, 0.0168, 0.0746, 0.0482, 0.1605, 0.0019, 0.0024, 0.0018,
        0.0963, 0.0330, 0.0203, 0.0112, 0.0246, 0.0047, 0.0054, 0.0605, 0.0128],
       grad_fn=<DivBackward0>)

In [117]:
ix = torch.multinomial(y_pred_prob,num_samples=1,replacement=True,generator=g).item()
ix

0

In [ ]:
# code for sample
for i in range(10):
    ix = stoi('.')
    str = []
    while True:        
        x_input = F.one_hot(torch.tensor(ix), len(chars)).float()
        y_pred_logits = x_input @ w
        y_pred_prob = torch.exp(y_pred_logits)/torch.sum(torch.exp(y_pred_logits))
        ix = torch.multinomial(y_pred_prob,num_samples=1,replacement=True).item()
        if ix == 0:
            print(''.join(str))
            break
        else:
            x = itos(ix)
            str.append(x)

jesh
ddeslahaleabilbemmah
wharsadheloueri
iagtowa
hn
kiomartirex
rtyrolianyanieriavahalerie
adh
ke
isicannaynaslukaia


In [72]:
# scale up the model
w1 = torch.randn(27,100,requires_grad=True)
b1 = torch.randn(100,requires_grad=True)
w2 = torch.randn(100,27,requires_grad=True)
b2 = torch.randn(27,requires_grad=True)

In [67]:
# all the data
g = torch.Generator().manual_seed(2147483647)
x = []
y = []
for w in words:
    chs = '.'+ w + '.'
    for c1, c2 in zip(chs, chs[1:]):
        x.append(stoi(c1))
        y.append(stoi(c2))
   
print(x[:5])
print(y[:5])

[0, 5, 13, 13, 1]
[5, 13, 13, 1, 0]


In [32]:
X = F.one_hot(torch.tensor(x[0]),num_classes=len(chars)).float()
h1 = X @ w1+b1
a1 =F.relu(h1)

In [33]:
logits = a1 @ w2+b2
logits

tensor([ -6.5700,   0.0390,   1.5548,  -5.3357,  14.5210,  11.1880, -12.2295,
         -0.8940,   6.7560,   4.2614, -22.1840,  -2.2490,  -6.8849,  -2.9322,
          6.9170,  14.9515,  13.5184,   0.4353,   5.6832,   6.5862,   0.4870,
        -14.0827,  -9.7621, -12.3189,  17.7748,  -1.2483,   1.8406],
       grad_fn=<AddBackward0>)

In [42]:
prob = torch.exp(logits) /(torch.sum(torch.exp(logits)))

-torch.log(torch.sum(prob * Y[0]))

tensor(6.6944, grad_fn=<NegBackward0>)

In [ ]:
# 分割训练集和测试集
split_size = 0.8
train_size = int(split_size * len(x))
train_data = x[:train_size]
train_target = y[:train_size]
test_data = x[train_size:]
test_target = y[train_size:]

In [44]:
X = F.one_hot(torch.tensor(x),num_classes=len(chars)).float()
Y = F.one_hot(torch.tensor(y),num_classes=len(chars)).float()
X.shape

torch.Size([228146, 27])

In [70]:
train_X = F.one_hot(torch.tensor(train_data),num_classes=len(chars)).float()
train_Y = F.one_hot(torch.tensor(train_target),num_classes=len(chars)).float()

test_X = F.one_hot(torch.tensor(test_data),num_classes=len(chars)).float()
test_Y = F.one_hot(torch.tensor(test_target),num_classes=len(chars)).float()

In [45]:
h1 = X @ w1+b1
a1 = F.relu(h1)
logits = a1 @ w2+b2
prob = torch.exp(logits) /(torch.sum(torch.exp(logits),dim=1,keepdim=True))
r_prob = prob * Y
r_prob_sum = torch.sum(r_prob, dim=1)
nll = -torch.log(r_prob_sum)
loss = torch.mean(nll)
print(loss)

tensor(17.4628, grad_fn=<MeanBackward0>)


In [24]:
lr = 1

In [ ]:
for i in range(500):
    h1 = train_X @ w1+b1
    a1 = F.relu(h1)
    logits = a1 @ w2+b2
    prob = torch.exp(logits) /(torch.sum(torch.exp(logits),dim=1,keepdim=True))
    r_prob = prob * train_Y
    r_prob_sum = torch.sum(r_prob, dim=1)
    nll = -torch.log(r_prob_sum)
    loss = torch.mean(nll)
    
    w1.grad = None
    w2.grad = None
    b1.grad = None
    b2.grad = None
    loss.backward()
    w1.data = w1.data - lr*w1.grad.data
    b1.data = b1.data - lr*b1.grad.data
    w2.data = w2.data - lr*w2.grad.data
    b2.data = b2.data - lr*b2.grad.data
    
    # 测试集上验证
    val_h1 = test_X @ w1+b1
    val_a1 = F.relu(val_h1)
    val_logits = val_a1 @ w2+b2
    val_prob = torch.exp(val_logits) /(torch.sum(torch.exp(val_logits),dim=1,keepdim=True))
    val_r_prob = val_prob * test_Y
    val_r_prob_sum = torch.sum(val_r_prob, dim=1)
    val_nll = -torch.log(val_r_prob_sum)
    val_loss = torch.mean(val_nll)
    
    print("loss:",loss,"val:",val_loss)

loss: tensor(13.5717, grad_fn=<MeanBackward0>) val: tensor(17.8173, grad_fn=<MeanBackward0>)
loss: tensor(16.6068, grad_fn=<MeanBackward0>) val: tensor(14.0709, grad_fn=<MeanBackward0>)
loss: tensor(14.2612, grad_fn=<MeanBackward0>) val: tensor(9.2267, grad_fn=<MeanBackward0>)
loss: tensor(9.0054, grad_fn=<MeanBackward0>) val: tensor(6.8852, grad_fn=<MeanBackward0>)
loss: tensor(6.2537, grad_fn=<MeanBackward0>) val: tensor(5.9650, grad_fn=<MeanBackward0>)
loss: tensor(5.4305, grad_fn=<MeanBackward0>) val: tensor(5.8563, grad_fn=<MeanBackward0>)
loss: tensor(5.2346, grad_fn=<MeanBackward0>) val: tensor(6.1336, grad_fn=<MeanBackward0>)
loss: tensor(5.5799, grad_fn=<MeanBackward0>) val: tensor(5.2780, grad_fn=<MeanBackward0>)
loss: tensor(5.0652, grad_fn=<MeanBackward0>) val: tensor(4.8277, grad_fn=<MeanBackward0>)
loss: tensor(4.3883, grad_fn=<MeanBackward0>) val: tensor(4.5882, grad_fn=<MeanBackward0>)
loss: tensor(4.1561, grad_fn=<MeanBackward0>) val: tensor(4.0623, grad_fn=<MeanBackwa

In [95]:
for i in range(20):
    ix = stoi('.')
    str = []
    while True:        
        x_input = F.one_hot(torch.tensor(ix), len(chars)).float()
        h1 = x_input @ w1 + b1
        a1 = F.relu(h1)
        y_pred_logits = a1 @ w2 + b2
        y_pred_prob = torch.exp(y_pred_logits)/torch.sum(torch.exp(y_pred_logits))
        ix = torch.multinomial(y_pred_prob,num_samples=1,replacement=True).item()
        if ix == 0:
            print(''.join(str))
            break
        else:
            x = itos(ix)
            str.append(x)

h
kyrlielalalanara
pha
s
mialliynnyvqlarylilaianiaya
blereri
brizalahry
mober
biskharore
ryn
ctarmeel
aroleiet
kakale
cahalten
osl
aziermsia
fe
keleth
lemiishiaan
jon
